In [ ]:
import os, sys, time, gc, random
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
device = torch.device("cuda:0")
print(f"Using device: {device}")

In [ ]:
sys.path.append(os.path.abspath(os.path.join('..', 'source')))
from CountNetTorch import CountNet, seed_everything
from metrics import compute_metrics

In [ ]:
path_data = os.path.abspath('data')
path_res  = os.path.abspath('results')
metrics = ['RMSP', 'RMD', 'R2']
n_folds = 10

In [ ]:
data_names = ['epilepsy', 'cd4', 'bolus', 'owls', 'fruits']
optimizer = 'Adam'
learning_rates = [0.01, 0.001]
batch_sizes = [32, 16]
num_nodes_list = [[8,8], [4,4]]
activation = torch.nn.LeakyReLU()
max_epochs = 200
patience = 10
sig2u_init = 0.8

### Poisson-NN(m)

In [ ]:
# ---------------------------------
model_name = 'PNN-M'
# ---------------------------------
is_conditional = False
for learning_rate in learning_rates:
    for batch_size in batch_sizes:
        for num_nodes in num_nodes_list:
            experiment = f'lr{learning_rate}-batch{batch_size}-nodes{num_nodes[0]}'
            results = {metric:np.zeros((n_folds, len(data_names))) for metric in metrics+['time']}
            for data_number, data_name in enumerate(data_names):
                data = pd.read_csv(f'{path_data}/{data_name}-prep.csv', dtype='float32', engine='pyarrow')        
                n_clusters = np.unique(data['id']).shape[0]
                n_features = 1+max([int(col_name[1:]) if 'x' in col_name else 0 for col_name in data.columns])
                for test_fold in tqdm(range(n_folds), desc=data_name):
                    valid_fold = (test_fold+1)//n_folds
                    data_dict = {
                        'train':data[-data['fold'].isin([valid_fold, test_fold])],
                        'valid':data[data['fold'] == valid_fold],
                        'test' :data[data['fold'] == test_fold],
                    }
                    for subset_name, subset in data_dict.items():
                        z = np.array(subset['id'], dtype=np.int32)
                        globals()[f'X_{subset_name}'] = np.array(subset[[f'x{i}' for i in range(n_features)]], dtype=np.float32)
                        globals()[f'Z_{subset_name}'] = np.zeros((len(z), n_clusters), dtype=np.float32)
                        globals()[f'Z_{subset_name}'][np.arange(len(z)), z] = 1
                        globals()[f'y_{subset_name}'] = np.array(subset['y'], dtype=np.float32).reshape(-1,1)
                    torch.cuda.empty_cache(); gc.collect()
                    seed_everything()
                    M = CountNet(n_clusters, n_features, num_nodes, activation, device=device)
                    start_time = time.time()
                    M.train_clik([y_train, X_train, Z_train], [y_valid, X_valid, Z_valid], is_conditional,
                        optimizer, learning_rate, batch_size, max_epochs, patience)
                    computing_time = time.time() - start_time
                    y_pred = M.predict(X_test)
                        offset_test = np.array(data_dict['test']['offset']).flatten()
                        y_pred = y_pred * offset_test
                        y_test = y_test * offset_test
                        y_train = y_train * np.array(data_dict['train']['offset']).flatten()
                    temp_results = compute_metrics(y_test, y_pred, np.mean(y_train), metrics)
                    for metric in metrics:
                        results[metric][test_fold, data_number] = temp_results[metric]
                    results['time'][test_fold, data_number] = computing_time
                    del M; gc.collect()
            for metric in metrics+['time']:
                pd.DataFrame(results[metric], columns=data_names).to_csv(f'{path_res}/{model_name}-{metric}-{experiment}.csv', index=False)

### Poisson-NN(c)

In [ ]:
# ---------------------------------
model_name = 'PNN-C'
# ---------------------------------
results = {metric:np.zeros((n_repeats*len(input_distributions), len(random_effects_types))) for metric in metrics+['time']}
experiments = [f'{input_distribution}-{simulation_number}' for input_distribution in input_distributions for simulation_number in range(n_repeats)]
is_conditional = True
for k_re, random_effects_type in enumerate(random_effects_types):    
    for k_in, input_distribution in enumerate(input_distributions):
        for simulation_number in tqdm(range(n_repeats), desc=f'X: {input_distribution}, u: {random_effects_type}'):
            data = pd.read_csv(f'{path_data}/data-{input_distribution}-{random_effects_type}-{simulation_number}.csv')
            data = {
                'train':data[data['number'].isin(range(cluster_size_train))],
                'valid':data[data['number'].isin(range(cluster_size_train, cluster_size_train+cluster_size_valid))],
                'test' :data[data['number'].isin(range(cluster_size_train+cluster_size_valid, cluster_size))],
            }
            for subset_name, subset in data.items():
                z = np.array(subset['cluster'], dtype=np.int32)
                globals()[f'y_{subset_name}'] = np.array(subset['y'], dtype=np.float32).reshape(-1,1)
                globals()[f'X_{subset_name}'] = np.array(subset[[f'x{i}' for i in range(n_features)]], dtype=np.float32)
                globals()[f'Z_{subset_name}'] = np.zeros((len(z), n_clusters), dtype=np.float32)
                globals()[f'Z_{subset_name}'][np.arange(len(z)), z] = 1                
            torch.cuda.empty_cache(); gc.collect()
            seed_everything()
            M = CountNet(n_clusters, n_features, num_nodes, activation, device=device)
            start_time = time.time()
            M.train_clik([y_train, X_train, Z_train], [y_valid, X_valid, Z_valid], is_conditional,
                optimizer, learning_rate, batch_size, max_epochs, patience)
            computing_time = time.time() - start_time
            y_pred = M.predict(X_test, Z_test)            
            temp_results = compute_metrics(y_test, y_pred, np.mean(y_train), metrics)
            for metric in metrics:
                results[metric][(k_in*n_repeats+simulation_number), k_re] = temp_results[metric]
            results['time'][(k_in*n_repeats+simulation_number), k_re] = computing_time
            del M; gc.collect()    
for metric in metrics+['time']:
    pd.DataFrame(results[metric], columns=random_effects_types, index=experiments).to_csv(f'{path_res}/{model_name}-{metric}.csv', index=True)

In [ ]:
# ---------------------------------
model_name = 'PNN-C'
# ---------------------------------
is_conditional = True
for learning_rate in learning_rates:
    for batch_size in batch_sizes:
        for num_nodes in num_nodes_list:
            experiment = f'lr{learning_rate}-batch{batch_size}-nodes{num_nodes[0]}'
            results = {metric:np.zeros((n_folds, len(data_names))) for metric in metrics+['time']}
            for data_number, data_name in enumerate(data_names):
                data = pd.read_csv(f'{path_data}/{data_name}-prep.csv', dtype='float32', engine='pyarrow')        
                n_clusters = np.unique(data['id']).shape[0]
                n_features = 1+max([int(col_name[1:]) if 'x' in col_name else 0 for col_name in data.columns])
                for test_fold in tqdm(range(n_folds), desc=data_name):
                    valid_fold = (test_fold+1)//n_folds
                    data_dict = {
                        'train':data[-data['fold'].isin([valid_fold, test_fold])],
                        'valid':data[data['fold'] == valid_fold],
                        'test' :data[data['fold'] == test_fold],
                    }
                    for subset_name, subset in data_dict.items():
                        z = np.array(subset['id'], dtype=np.int32)
                        globals()[f'X_{subset_name}'] = np.array(subset[[f'x{i}' for i in range(n_features)]], dtype=np.float32)
                        globals()[f'Z_{subset_name}'] = np.zeros((len(z), n_clusters), dtype=np.float32)
                        globals()[f'Z_{subset_name}'][np.arange(len(z)), z] = 1
                        globals()[f'y_{subset_name}'] = np.array(subset['y'], dtype=np.float32).reshape(-1,1)
                    torch.cuda.empty_cache(); gc.collect()
                    seed_everything()
                    M = CountNet(n_clusters, n_features, num_nodes, activation, device=device)
                    start_time = time.time()
                    M.train_clik([y_train, X_train, Z_train], [y_valid, X_valid, Z_valid], is_conditional,
                        optimizer, learning_rate, batch_size, max_epochs, patience)
                    computing_time = time.time() - start_time
                    y_pred = M.predict(X_test, Z_test)
                    if data_name == 'owls':
                        offset_test = np.array(data_dict['test']['offset']).flatten()
                        y_pred = y_pred * offset_test
                        y_test = y_test * offset_test
                        y_train = y_train * np.array(data_dict['train']['offset']).flatten()
                    temp_results = compute_metrics(y_test, y_pred, np.mean(y_train), metrics)
                    for metric in metrics:
                        results[metric][test_fold, data_number] = temp_results[metric]
                    results['time'][test_fold, data_number] = computing_time
                    del M; gc.collect()
            for metric in metrics+['time']:
                pd.DataFrame(results[metric], columns=data_names).to_csv(f'{path_res}/{model_name}-{metric}-{experiment}.csv', index=False)

### PGNN

In [ ]:
# ---------------------------------
model_name = 'PGNN'
# ---------------------------------
results = {metric:np.zeros((n_repeats*len(input_distributions), len(random_effects_types))) for metric in metrics+['time']}
experiments = [f'{input_distribution}-{simulation_number}' for input_distribution in input_distributions for simulation_number in range(n_repeats)]
for k_re, random_effects_type in enumerate(random_effects_types):
    for k_in, input_distribution in enumerate(input_distributions):
        for simulation_number in tqdm(range(n_repeats), desc=f'X: {input_distribution}, u: {random_effects_type}'):
            data = pd.read_csv(f'{path_data}/data-{input_distribution}-{random_effects_type}-{simulation_number}.csv')
            data = {
                'train':data[data['number'].isin(range(cluster_size_train))],
                'valid':data[data['number'].isin(range(cluster_size_train, cluster_size_train+cluster_size_valid))],
                'test' :data[data['number'].isin(range(cluster_size_train+cluster_size_valid, cluster_size))],
            }
            for subset_name, subset in data.items():
                z = np.array(subset['cluster'], dtype=np.int32)
                globals()[f'y_{subset_name}'] = np.array(subset['y'], dtype=np.float32).reshape(-1,1)
                globals()[f'X_{subset_name}'] = np.array(subset[[f'x{i}' for i in range(n_features)]], dtype=np.float32)
                globals()[f'Z_{subset_name}'] = np.zeros((len(z), n_clusters), dtype=np.float32)
                globals()[f'Z_{subset_name}'][np.arange(len(z)), z] = 1                
            torch.cuda.empty_cache(); gc.collect()
            seed_everything()
            M = CountNet(n_clusters, n_features, num_nodes, activation, device=device)
            start_time = time.time()
            M.train_hlik([y_train, X_train, Z_train], [y_valid, X_valid, Z_valid],
                optimizer, learning_rate, batch_size, max_epochs, patience, sig2u_init)
            computing_time = time.time() - start_time
            y_pred = M.predict(X_test, Z_test)
            temp_results = compute_metrics(y_test, y_pred, np.mean(y_train), metrics)
            for metric in metrics:
                results[metric][(k_in*n_repeats+simulation_number), k_re] = temp_results[metric]
            results['time'][(k_in*n_repeats+simulation_number), k_re] = computing_time
            file_history = f'{os.path.abspath('model')}/{model_name}-{input_distribution}-{random_effects_type}-{simulation_number}-history.pkl'
            with open(file_history, 'wb') as f: pickle.dump(M.history, f)
            del M; gc.collect()
for metric in metrics+['time']:
    pd.DataFrame(results[metric], columns=random_effects_types, index=experiments).to_csv(f'{path_res}/{model_name}-{metric}.csv', index=True)

In [ ]:
# ---------------------------------
model_name = 'HLMMNN'
# ---------------------------------
for learning_rate in learning_rates:
    for batch_size in batch_sizes:
        for num_nodes in num_nodes_list:
            experiment = f'lr{learning_rate}-batch{batch_size}-nodes{num_nodes[0]}'
            results = {metric:np.zeros((n_folds, len(data_names))) for metric in metrics+['time']}
            for data_number, data_name in enumerate(data_names):
                data = pd.read_csv(f'{path_data}/{data_name}-prep.csv', dtype='float32', engine='pyarrow')        
                n_clusters = np.unique(data['id']).shape[0]
                n_features = 1+max([int(col_name[1:]) if 'x' in col_name else 0 for col_name in data.columns])
                for test_fold in tqdm(range(n_folds), desc=data_name):
                    valid_fold = (test_fold+1)//n_folds
                    data_dict = {
                        'train':data[-data['fold'].isin([valid_fold, test_fold])],
                        'valid':data[data['fold'] == valid_fold],
                        'test' :data[data['fold'] == test_fold],
                    }
                    for subset_name, subset in data_dict.items():
                        z = np.array(subset['id'], dtype=np.int32)
                        globals()[f'X_{subset_name}'] = np.array(subset[[f'x{i}' for i in range(n_features)]], dtype=np.float32)
                        globals()[f'Z_{subset_name}'] = np.zeros((len(z), n_clusters), dtype=np.float32)
                        globals()[f'Z_{subset_name}'][np.arange(len(z)), z] = 1
                        globals()[f'y_{subset_name}'] = np.array(subset['y'], dtype=np.float32).reshape(-1,1)
                    torch.cuda.empty_cache(); gc.collect()
                    seed_everything()
                    M = CountNet(n_clusters, n_features, num_nodes, activation, device=device)
                    start_time = time.time()
                    M.train_hlik([y_train, X_train, Z_train], [y_valid, X_valid, Z_valid],
                        optimizer, learning_rate, batch_size, max_epochs, patience, sig2u_init)
                    computing_time = time.time() - start_time
                    y_pred = M.predict(X_test, Z_test)
                    if data_name == 'owls':
                        offset_test = np.array(data_dict['test']['offset']).flatten()
                        y_pred = y_pred * offset_test
                        y_test = y_test * offset_test
                        y_train = y_train * np.array(data_dict['train']['offset']).flatten()
                    temp_results = compute_metrics(y_test, y_pred, np.mean(y_train), metrics)
                    for metric in metrics:
                        results[metric][test_fold, data_number] = temp_results[metric]
                    results['time'][test_fold, data_number] = computing_time
                    del M; gc.collect()
            for metric in metrics+['time']:
                pd.DataFrame(results[metric], columns=data_names).to_csv(f'{path_res}/{model_name}-{metric}-{experiment}.csv', index=False)